In [1]:
import pandas as pd
df = pd.read_csv("stats/summary_sampling.csv")
df

,questionnaire,model,mode_mode,test_count,Extraversion_mean,Extraversion_std,Agreeableness_mean,Agreeableness_std,Conscientiousness_mean,Conscientiousness_std,Neuroticism_mean,Neuroticism_std,Openness_mean,Openness_std
0,BFI,llama3_70b,prompt_chat_sampling_1xxxx,5,2.3,0.1,3.6,0.3,3.6,0.1,2.8,0.2,2.3,0.2
1,IPIP-NEO,llama3_70b,prompt_chat_sampling_0xxxx,5,3.9,0.1,4.0,0.1,3.7,0.0,2.3,0.0,3.5,0.0
2,BFI,llama3_70b,prompt_chat_sampling_x0xxx,5,3.5,0.1,4.2,0.1,4.1,0.1,2.2,0.2,3.5,0.1
3,BFI,llama3_70b,prompt_chat_sampling_x1xxx,5,3.8,0.1,3.7,0.1,2.3,0.1,2.7,0.1,3.3,0.1
4,BFI,llama3_70b,prompt_chat_sampling_xx0xx,5,4.3,0.2,4.3,0.1,3.8,0.1,2.2,0.1,3.9,0.1
5,IPIP-NEO,llama3_70b,prompt_chat_sampling_1xxxx,5,2.2,0.1,3.5,0.1,3.4,0.1,2.7,0.1,2.6,0.1
6,BFI,llama3_70b,prompt_chat_sampling_xx1xx,5,2.4,0.1,4.2,0.1,3.8,0.1,2.6,0.2,3.4,0.1
7,BFI,llama3_70b,prompt_chat_sampling_xxx0x,5,4.1,0.1,4.4,0.1,3.9,0.1,2.1,0.1,3.8,0.1
8,BFI,llama3_70b,prompt_chat_sampling_xxx1x,5,3.2,0.1,1.8,0.2,3.3,0.1,3.2,0.2,2.6,0.1
9,IPIP-NEO,llama3_70b,prompt_chat_sampling_x0xxx,5,3.4,0.1,4.0,0.1,4.0,0.0,2.2,0.1,3.0,0.1


In [3]:
labels = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']
for idx_label, label in enumerate(labels):
    df[f"{label}"] = df[f"{label}_mean"].astype(str) + r" $\pm$ " + df[f"{label}_std"].astype(str)
    df = df.drop(columns=[f"{label}_mean", f"{label}_std"])

In [4]:
# llama3 8b
data_dict = {
    "Sampling": []
}

def find_non_x_index(string):
    # Iterate through the string to find the index where the character is not 'x'
    for i, char in enumerate(string):
        if char != 'x':
            return i
    # If no such index exists, you can raise an error or return a special value
    raise ValueError("No character other than 'x' found in the string.")

data = []
for questionnaire in ['BFI', 'IPIP-NEO']:
    for model, model_abbr in [['LLaMA3-70B', 'llama3_70b']]:
        for method, method_abbr in [['Sampling', 'prompt_chat_sampling_']]:
            if method_abbr == "prompt_chat_" and model_abbr == "llama3_8b":
                continue
            row = [questionnaire, model, method]
            
            row_avg_high, row_std_high = 0, 0
            row_avg_low, row_std_low = 0, 0
            for mode in ["0xxxx", '1xxxx', 'x0xxx', 'x1xxx', 'xx0xx', 'xx1xx', 'xxx0x', 'xxx1x', 'xxxx0', 'xxxx1']:
                label = labels[find_non_x_index(mode)]
                val = df.loc[((df['questionnaire'] == questionnaire) & (df['mode_mode'] == f'{method_abbr}{mode}')) & (df['model'] == model_abbr)][[label]].values.tolist()[0][0]
                row.append(val)
                val_avg, val_std = val.split(r" $\pm$ ")
                if "0" in mode:
                    row_avg_high += float(val_avg)
                    row_std_high += float(val_std)
                elif "1" in mode:
                    row_avg_low += float(val_avg)
                    row_std_low += float(val_std)
            row.append(fr"{row_avg_high/5:.1f} $\pm$ {row_std_high/5:.1f}")
            row.append(fr"{row_avg_low/5:.1f} $\pm$ {row_std_low/5:.1f}")
            data.append(row)
df_1 = pd.DataFrame(data)
df_1.columns = ['Questionnaire', 'Model', 'Method'] + [f"{level} {label}" for label in labels for level in ['High', 'Low']] + ["High Average", "Low Average"]

In [6]:
print(df_1.to_latex())

\begin{tabular}{llllllllllllllll}
\toprule
 & Questionnaire & Model & Method & High Openness & Low Openness & High Conscientiousness & Low Conscientiousness & High Extraversion & Low Extraversion & High Agreeableness & Low Agreeableness & High Neuroticism & Low Neuroticism & High Average & Low Average \\
\midrule
0 & BFI & LLaMA3-70B & Sampling & 4.4 $\pm$ 0.1 & 2.3 $\pm$ 0.2 & 4.1 $\pm$ 0.1 & 2.3 $\pm$ 0.1 & 4.3 $\pm$ 0.2 & 2.4 $\pm$ 0.1 & 4.4 $\pm$ 0.1 & 1.8 $\pm$ 0.2 & 3.5 $\pm$ 0.1 & 2.1 $\pm$ 0.2 & 4.1 $\pm$ 0.1 & 2.2 $\pm$ 0.2 \\
1 & IPIP-NEO & LLaMA3-70B & Sampling & 3.5 $\pm$ 0.0 & 2.6 $\pm$ 0.1 & 4.0 $\pm$ 0.0 & 2.6 $\pm$ 0.1 & 4.0 $\pm$ 0.1 & 2.5 $\pm$ 0.1 & 4.3 $\pm$ 0.0 & 2.1 $\pm$ 0.1 & 3.0 $\pm$ 0.1 & 2.3 $\pm$ 0.1 & 3.8 $\pm$ 0.0 & 2.4 $\pm$ 0.1 \\
\bottomrule
\end{tabular}

